In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
import os, sys

In [3]:
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))

In [4]:
%matplotlib inline
import utils; reload(utils);
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [5]:
path = "data/ml-20m/"
#path = "data/ml-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

## Setup data

In [6]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [7]:
len(ratings)

20000263

In [8]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [9]:
movie_names

{131072: 'Jesus liebt mich (2012)',
 1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (Cit\xc3\xa9 des enfants perdus, La) (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 3

In [10]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [11]:
userId_revIndex = {id: i for i, id in enumerate(users)}
movieId_revIndex = {id: i for i, id in enumerate(movies)}

In [12]:
userId_revIndex

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 11

In [13]:
movieId_revIndex

{131072: 23878,
 1: 227,
 2: 0,
 3: 175,
 4: 1994,
 5: 1044,
 6: 364,
 7: 420,
 8: 1965,
 9: 1677,
 10: 365,
 11: 386,
 12: 2614,
 13: 4834,
 14: 1187,
 15: 429,
 16: 430,
 17: 387,
 18: 1469,
 19: 366,
 20: 1678,
 21: 630,
 22: 1646,
 23: 1679,
 24: 228,
 25: 681,
 26: 3537,
 27: 3816,
 28: 2021,
 29: 1,
 30: 3163,
 31: 1647,
 32: 2,
 33: 10410,
 34: 1045,
 35: 3809,
 36: 1046,
 37: 7844,
 38: 3553,
 39: 631,
 40: 3538,
 41: 2123,
 42: 1680,
 43: 2074,
 44: 1681,
 45: 1867,
 46: 1682,
 47: 3,
 48: 632,
 49: 7031,
 50: 4,
 51: 8381,
 52: 421,
 53: 10411,
 54: 4837,
 55: 1311,
 56: 8382,
 57: 1683,
 58: 1868,
 59: 3164,
 60: 388,
 61: 1312,
 62: 176,
 63: 2615,
 64: 2050,
 65: 688,
 66: 3505,
 67: 1313,
 68: 1884,
 69: 1396,
 70: 177,
 71: 4838,
 72: 4149,
 73: 1470,
 74: 1314,
 75: 3554,
 76: 1315,
 77: 4977,
 78: 2124,
 79: 1316,
 80: 5579,
 81: 2616,
 82: 3165,
 83: 3166,
 84: 3167,
 85: 1995,
 86: 1317,
 87: 3555,
 88: 1318,
 89: 1471,
 90: 3902,
 92: 1319,
 93: 1684,
 94: 2075,
 95

In [14]:
ratings.movieId = ratings.movieId.apply(lambda x: movieId_revIndex[x])
ratings.userId = ratings.userId.apply(lambda x: userId_revIndex[x])

In [15]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

In [18]:
n_factors = 50; # number of embedding parameters for each user and movie
np.random.seed = 42

In [19]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

## Basic model - dot product

In [20]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
user_embedding = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in) # what is W_regularizer
movie_embedding = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [21]:
x = merge([user_embedding, movie_embedding], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse', metrics=['acc'])


In [22]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 50)         6924650     user_in[0][0]                    
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 50)         1337200     movie_in[0][0]                   
___________________________________________________________________________________________

In [ ]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3,
          validation_data=([val.userId, val.movieId], val.rating))

In [54]:
model.optimizer.lr = 0.0001

In [55]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79953 samples, validate on 20051 samples
Epoch 1/3
79953/79953 [==============================] - 6s - loss: 2.1216 - acc: 0.2981 - val_loss: 1.4125 - val_acc: 0.2900
Epoch 2/3
79953/79953 [==============================] - 6s - loss: 2.1000 - acc: 0.2993 - val_loss: 1.4145 - val_acc: 0.2845
Epoch 3/3
79953/79953 [==============================] - 6s - loss: 2.0791 - acc: 0.3035 - val_loss: 1.4114 - val_acc: 0.2844


## Add bias terms

In [56]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
user_embedding = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in) # what is W_regularizer
movie_embedding = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)
user_bias = Embedding(n_users, 1, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_bias = Embedding(n_movies, 1, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [64]:
user_bias = Flatten()(user_bias)
movie_bias = Flatten()(movie_bias)
# May be ok to not use Flatten as output is 1?
# Shape won't be the same as user_embedding

In [66]:
x = merge([user_embedding, movie_embedding], mode='dot')
x = Flatten()(x)
x = merge([user_bias, x], mode='sum')
x = merge([movie_bias, x], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse', metrics=['accuracy'])

In [67]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79953 samples, validate on 20051 samples
Epoch 1/3
79953/79953 [==============================] - 6s - loss: 8.8318 - acc: 0.0444 - val_loss: 3.0987 - val_acc: 0.1562
Epoch 2/3
79953/79953 [==============================] - 6s - loss: 2.6104 - acc: 0.2219 - val_loss: 1.6064 - val_acc: 0.2621
Epoch 3/3
79953/79953 [==============================] - 6s - loss: 2.0289 - acc: 0.2779 - val_loss: 1.3184 - val_acc: 0.2849


In [68]:
model.optimizer.lr = 0.01

In [69]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79953 samples, validate on 20051 samples
Epoch 1/3
79953/79953 [==============================] - 6s - loss: 1.8809 - acc: 0.2957 - val_loss: 1.2120 - val_acc: 0.2926
Epoch 2/3
79953/79953 [==============================] - 6s - loss: 1.8048 - acc: 0.3034 - val_loss: 1.1657 - val_acc: 0.2940
Epoch 3/3
79953/79953 [==============================] - 6s - loss: 1.7465 - acc: 0.3062 - val_loss: 1.1313 - val_acc: 0.2976


In [73]:
model.optimizer.lr = 0.001
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=10,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79953 samples, validate on 20051 samples
Epoch 1/10
79953/79953 [==============================] - 6s - loss: 1.2624 - acc: 0.4254 - val_loss: 0.9217 - val_acc: 0.3119
Epoch 2/10
79953/79953 [==============================] - 6s - loss: 1.2518 - acc: 0.4291 - val_loss: 0.9154 - val_acc: 0.3135
Epoch 3/10
79953/79953 [==============================] - 6s - loss: 1.2423 - acc: 0.4354 - val_loss: 0.9119 - val_acc: 0.3142
Epoch 4/10
79953/79953 [==============================] - 6s - loss: 1.2329 - acc: 0.4398 - val_loss: 0.9089 - val_acc: 0.3129
Epoch 5/10
79953/79953 [==============================] - 6s - loss: 1.2255 - acc: 0.4431 - val_loss: 0.9044 - val_acc: 0.3163
Epoch 6/10
79953/79953 [==============================] - 6s - loss: 1.2187 - acc: 0.4451 - val_loss: 0.8999 - val_acc: 0.3137
Epoch 7/10
79953/79953 [==============================] - 6s - loss: 1.2119 - acc: 0.4503 - val_loss: 0.8992 - val_acc: 0.3120
Epoch 8/10
79953/79953 [==============================] - 6s 

In [74]:
model.save_weights(path+'bias.h5')
model.load_weights(path+'bias.h5')

In [75]:
model.predict([np.array([100]), np.array([200])])

array([[ 3.1772]], dtype=float32)

## neural net

In [24]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
user_embedding = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in) # W_regularizer is l1/l2 regularization.
movie_embedding = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [25]:
x = merge([user_embedding, movie_embedding], mode='concat')
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(1)(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse', metrics=['accuracy'])

In [26]:
model.optimizer.lr = 0.001
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 16002195 samples, validate on 3998068 samples
Epoch 1/1
 7089728/16002195 [============>.................] - ETA: 3491s - loss: 0.9213 - acc: 0.3196

KeyboardInterrupt: 

In [27]:
model.save_weights(path+'nn.h5')
model.load_weights(path+'nn.h5')

In [28]:
model.predict([np.array([100]), np.array([200])])

array([[ 3.0176]], dtype=float32)